Task 2:
Predicting customer buying behaviour

Importing Libraries

In [5]:
%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings("ignore")

Data Importing

In [8]:
data= 'customer_booking.csv'

In [10]:
import chardet

with open('customer_booking.csv', 'rb') as rawdata:
  result = chardet.detect(rawdata.read(100000))

print(result)

{'encoding': 'ISO-8859-1', 'confidence': 0.73, 'language': ''}


In [11]:
df= pd.read_csv(data, encoding='ISO-8859-1')

In [12]:
df.head()

num_passengers sales_channel  trip_type  purchase_lead  length_of_stay   
0               2      Internet  RoundTrip            262              19  \
1               1      Internet  RoundTrip            112              20   
2               2      Internet  RoundTrip            243              22   
3               1      Internet  RoundTrip             96              31   
4               2      Internet  RoundTrip             68              22   

   flight_hour flight_day   route booking_origin  wants_extra_baggage   
0            7        Sat  AKLDEL    New Zealand                    1  \
1            3        Sat  AKLDEL    New Zealand                    0   
2           17        Wed  AKLDEL          India                    1   
3            4        Sat  AKLDEL    New Zealand                    0   
4           15        Wed  AKLDEL          India                    1   

   wants_preferred_seat  wants_in_flight_meals  flight_duration   
0                     0                      0             5.52  \
1                     0                      0             5.52   
2                     1                      0             5.52   
3                     0                      1             5.52   
4                     0                      1             5.52   

   booking_complete  
0                 0  
1                 0  
2                 0  
3                 0  
4                 0

Data Analysis

In [13]:
df.dtypes


num_passengers             int64
sales_channel             object
trip_type                 object
purchase_lead              int64
length_of_stay             int64
flight_hour                int64
flight_day                object
route                     object
booking_origin            object
wants_extra_baggage        int64
wants_preferred_seat       int64
wants_in_flight_meals      int64
flight_duration          float64
booking_complete           int64
dtype: object

In [14]:
df.shape

(50000, 14)

In [15]:
df.isnull().sum()

num_passengers           0
sales_channel            0
trip_type                0
purchase_lead            0
length_of_stay           0
flight_hour              0
flight_day               0
route                    0
booking_origin           0
wants_extra_baggage      0
wants_preferred_seat     0
wants_in_flight_meals    0
flight_duration          0
booking_complete         0
dtype: int64

In [16]:
df.booking_complete.value_counts()

booking_complete
0    42522
1     7478
Name: count, dtype: int64

In [17]:
X= df.drop('booking_complete',axis=1)
y= df.booking_complete         

for colname in X.select_dtypes("object"):
    X[colname], _ = X[colname].factorize()

In [18]:
X.dtypes

num_passengers             int64
sales_channel              int64
trip_type                  int64
purchase_lead              int64
length_of_stay             int64
flight_hour                int64
flight_day                 int64
route                      int64
booking_origin             int64
wants_extra_baggage        int64
wants_preferred_seat       int64
wants_in_flight_meals      int64
flight_duration          float64
dtype: object

In [19]:
from sklearn.feature_selection import mutual_info_classif

mi_scores = mutual_info_classif(X, y)
mi_scores = pd.Series(mi_scores, name="MI Scores", index=X.columns)
mi_scores = mi_scores.sort_values(ascending=False)

mi_scores 

route                    0.052376
booking_origin           0.047079
flight_duration          0.018703
length_of_stay           0.010456
wants_extra_baggage      0.009799
purchase_lead            0.004507
wants_in_flight_meals    0.003788
wants_preferred_seat     0.002301
flight_hour              0.002259
num_passengers           0.001778
flight_day               0.001508
trip_type                0.001398
sales_channel            0.000384
Name: MI Scores, dtype: float64

Data Visualization

In [20]:
def plot_mi_scores(scores):
    scores = scores.sort_values(ascending=True)
    width = np.arange(len(scores))
    ticks = list(scores.index)
    plt.barh(width, scores)
    plt.yticks(width, ticks)
    plt.title("Mutual Information Scores")


plt.figure(dpi=100, figsize=(8, 5))
plot_mi_scores(mi_scores)

sns.jointplot(data=df,x='route',y='')
We can see

route
booking_origin
flight_duration
wants_extra_baggage
length_of_stay

are the top 5 features which are dependant with booking_complete feature

Data Splitting

In [22]:
from sklearn.model_selection import train_test_split

def dataset(X,y):
    train_full_X, val_X, train_full_y, val_y = train_test_split(X, y,test_size=0.2,random_state = 0)

    train_X, test_X, train_y, test_y = train_test_split(train_full_X, train_full_y, test_size=0.25,random_state = 0)
    return (train_X, val_X, train_y, val_y)

In [23]:
from sklearn.preprocessing import MinMaxScaler

def scale(X):
    scaler = MinMaxScaler()
    scaler.fit(X)
    return X

Model Implementations

1. Random forest classifier with top six features

In [24]:
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier

features=['route','booking_origin','flight_duration','wants_extra_baggage', 'length_of_stay','num_passengers']
X= df[features]

X = pd.get_dummies(X, columns=features)
X= scale(X)
y= df.booking_complete       

X_train,X_val,y_train,y_val= dataset(X,y)

forest_model= RandomForestClassifier(random_state=1)
forest_model.fit(X_train, y_train)
preds= forest_model.predict(X_val)

print('ACCURACY: ',accuracy_score(y_val,preds)*100)
print('AUC score: ',roc_auc_score(y_val,preds))

ACCURACY:  83.36
AUC score:  0.5657818407546988


2. Random forest classifier with all features

In [25]:
X= df.drop('booking_complete',axis=1)

X = pd.get_dummies(X)
X= scale(X)
y= df.booking_complete       

X_train,X_val,y_train,y_val= dataset(X,y)

forest_model= RandomForestClassifier(random_state=1)
forest_model.fit(X_train, y_train)
preds= forest_model.predict(X_val)

print('ACCURACY: ',accuracy_score(y_val,preds)*100)
print('AUC score: ',roc_auc_score(y_val,preds))

ACCURACY:  84.76
AUC score:  0.5479604084813514


In [27]:
X

num_passengers  purchase_lead  length_of_stay  flight_hour   
0                   2            262              19            7  \
1                   1            112              20            3   
2                   2            243              22           17   
3                   1             96              31            4   
4                   2             68              22           15   
...               ...            ...             ...          ...   
49995               2             27               6            9   
49996               1            111               6            4   
49997               1             24               6           22   
49998               1             15               6           11   
49999               1             19               6           10   

       wants_extra_baggage  wants_preferred_seat  wants_in_flight_meals   
0                        1                     0                      0  \
1                        0                     0                      0   
2                        1                     1                      0   
3                        0                     0                      1   
4                        1                     0                      1   
...                    ...                   ...                    ...   
49995                    1                     0                      1   
49996                    0                     0                      0   
49997                    0                     0                      1   
49998                    1                     0                      1   
49999                    0                     1                      0   

       flight_duration  sales_channel_Internet  sales_channel_Mobile  ...   
0                 5.52                    True                 False  ...  \
1                 5.52                    True                 False  ...   
2                 5.52                    True                 False  ...   
3                 5.52                    True                 False  ...   
4                 5.52                    True                 False  ...   
...                ...                     ...                   ...  ...   
49995             5.62                    True                 False  ...   
49996             5.62                    True                 False  ...   
49997             5.62                    True                 False  ...   
49998             5.62                    True                 False  ...   
49999             5.62                    True                 False  ...   

       booking_origin_Timor-Leste  booking_origin_Tonga   
0                           False                 False  \
1                           False                 False   
2                           False                 False   
3                           False                 False   
4                           False                 False   
...                           ...                   ...   
49995                       False                 False   
49996                       False                 False   
49997                       False                 False   
49998                       False                 False   
49999                       False                 False   

       booking_origin_Tunisia  booking_origin_Turkey  booking_origin_Ukraine   
0                       False                  False                   False  \
1                       False                  False                   False   
2                       False                  False                   False   
3                       False                  False                   False   
4                       False                  False                   False   
...                       ...                    ...                     ...   
49995                   False                  False                   False   
499

3. XGB classifier with top six features

In [29]:
from xgboost import XGBClassifier

X= df[features]

X = pd.get_dummies(X, columns=features)
X= scale(X)

y= df.booking_complete    

X_train,X_val,y_train,y_val= dataset(X,y)
xgb_model = XGBClassifier()

xgb_model.fit(X_train, y_train)
prediction_xgb = xgb_model.predict(X_val)
print('ACCURACY: ',accuracy_score(y_val, prediction_xgb)*100)
print('AUC score: ',roc_auc_score(y_val,prediction_xgb))

ACCURACY:  84.78999999999999
AUC score:  0.5231540310747802


4. XGB classifier with all features

In [30]:
X= df.drop('booking_complete',axis=1)

X = pd.get_dummies(X)
X= scale(X)
y= df.booking_complete 

X_train,X_val,y_train,y_val= dataset(X,y)


xgb_model = XGBClassifier()
xgb_model.fit(X_train, y_train)
prediction_xgb = xgb_model.predict(X_val)
print('ACCURACY: ',accuracy_score(y_val, prediction_xgb)*100)
print('AUC score: ',roc_auc_score(y_val,prediction_xgb))

ACCURACY:  84.96000000000001
AUC score:  0.5431644571912829


Comparing the accuracy and auc scores promoting random forest with all features as the final model

In [31]:
X= df.drop('booking_complete',axis=1)

X = pd.get_dummies(X)
X= scale(X)
y= df.booking_complete       

train_full_X,test_X, train_full_y, test_y = train_test_split(X, y,test_size=0.2,random_state = 0)

forest_model= RandomForestClassifier(random_state=1)
forest_model.fit(train_full_X, train_full_y)
preds= forest_model.predict(test_X)

print('ACCURACY: ',accuracy_score(test_y,preds)*100)
print('AUC score: ',roc_auc_score(test_y,preds))

ACCURACY:  85.09
AUC score:  0.5577796717361984
